# Generate Market Book by Limit (MBL) limits and trades using dedicated service [Full Snapshot]  - C#

### Overview
Market Book by Price (MBP) also known as Market Book by Limit (MBL), is the price-based data of the book.
MBL restricts updates to a maximum of specified depth price levels and consolidates all the quantity (size) into a single update for each price level, which includes the total quantity (size).

**Note : This sample is not about Market Book by Order (MBO). For the MBO please refer the notebook [Tick] quotes**

Ganymede can translate streams in the MBO (Market By Order) format to MBL/MBP (Market By Limit, or Market By Price) order book data. 
In this example we use two well known ICE data sources for Euronext
1. One contributes full order book in MBO format (e.g: flat quotes)
2. The other contributes trades

The MBO flat quoted data is on-the-fly translated to the more easily usable MBL/MBP data (matrix)

For more information on MBL/MBP and MBO you can for example read [here](https://www.cmegroup.com/education/market-by-order-mbo.html)

### Inputs/outputs
Trades and book extraction sample requires several inputs :
* instrument identifier
* book depth 
* book update type: in this use-case, the update type is **snapshot only**.
* date and time constraints

### Services used
This sample uses *gRPC requests* in order to retrieve trades and book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and book tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run the MBL full snapshot sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.32.*"

In [ ]:
using System.IO;
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;

using TickTradesAndBookService = Systemathics.Apis.Services.Tick.V1.TickTradesAndBookService;
using TickTradesAndBookRequest = Systemathics.Apis.Services.Tick.V1.TickTradesAndBookRequest;
using TickTradesAndBookResponse = Systemathics.Apis.Services.Tick.V1.TickTradesAndBookResponse;
using Trade = Systemathics.Apis.Type.Shared.V1.Trade;
using Book = Systemathics.Apis.Type.Shared.V1.Book;
using BookUpdates = Systemathics.Apis.Type.Shared.V1.BookUpdates;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = new Grpc.Core.Metadata();
try { headers = TokenHelpers.GetTokenAsMetaData(); } catch {}

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection
* Depth
* Book update type

#### 3.1 Instrument and max depth selection

In [ ]:
// Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
// The ICE ticker
var ticker = "E:BNP";
// The book limit : get the top 10 best limits
var maxDepth = 10;
// The book update type: snapshot only or first snapshot then incremental
var updatesType = BookUpdates.SnapshotsOnly;

#### 3.2 Time period delimitation

Here we select a day and an intraday window of 1 minute width

In [ ]:
// Create time intervals
// Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
var start = new DateTime(2021, 11, 01);
var end = new DateTime(2021, 11, 01);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the tick quotes request time interval (we are using Google date time format)
// UTC time zone
var timeInterval = new TimeInterval()
{
     StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 51 },
     EndTime = new TimeOfDay { Hours = 14, Minutes = 01, Seconds = 00 } 
};

The following code snippet create request for *trades* and *book* data and instantiates the service client

In [ ]:
// Generate constraints based on the previous time selection
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

// Generate the tick and book request
var request = new TickTradesAndBookRequest { 
    Constraints = constraints, 
    MaxDepth = maxDepth, 
    BookUpdates = updatesType,
    Adjustment = false
 };

// We want to stream the ticks from those two Euronext sources (specific to provider "ICE")
request.Identifiers.Add(new IdentifierAndLevel { Provider = "ICE", Ticker = ticker, Exchange = "EQUITY_L2_973" /* this source is MBO/FOB Euronext */, Level = Level.TradesAndBook });
request.Identifiers.Add(new IdentifierAndLevel { Provider = "ICE", Ticker = ticker, Exchange = "787" /* this source is trades only Euronext */, Level = Level.Trades });

// Instantiate the tick quotes service client
var serviceClient = new TickTradesAndBookService.TickTradesAndBookServiceClient(channel);

### Step 4: Retrieve data

#### 4.1 Retrieve book & trades

We basically receive 2 types of data in the reponse
1. Trade data (price, size, trade id and trade condition)
2. Book data (a list of up to *maxDepth* limits on booth Bid and Ask sides of the book)

The following code exports to a file the time increasing trade (scalar) or book data (matrix)

In [ ]:
// Get the book and trades and export data into text file
var call = serviceClient.TickTradesAndBook(request, headers);
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
using (var t = new StreamWriter(File.OpenWrite($"output/{ticker}_mbl_full_snapshot.txt")))
await foreach (var item in call.ResponseStream.ReadAllAsync())
{
    if (item.Mapping != null)
    {
        var mapping = item.Mapping;
        foreach(var a in mapping.Values)
        {
            t.WriteLine($"# {a.Value} => {a.Identifier.Provider}/{a.Identifier.Exchange}/{a.Identifier.Ticker}");
        }

        continue;
    }

    var ts = item.Data.TimeStamp.ToDateTime();

    if (item.Data.Book != null)
    {
        var book = item.Data.Book;   
        var header = $"{ts:O} [{item.Data.Mapping}] [BOOK]";
        t.WriteLine();
        t.WriteLine(header);
        // In snapshot mode, the book limits are sorted in the natural order
        // This code leverage the fact we know that bids/asks always come complete/ordered, it wouldn't work in incremental updates mode
        var maxDepth = Math.Max(book.Bid.Count, book.Ask.Count);
        for (var depth = 0; depth < maxDepth; depth++)
        {
            var bidPrice = depth < book.Bid.Count ? $"{book.Bid[depth].Price:N4}" : string.Empty;
            var bidSize = depth < book.Bid.Count ? $"{book.Bid[depth].Size:N0}" : string.Empty;
            var askPrice = depth < book.Ask.Count ? $"{book.Ask[depth].Price:N4}" : string.Empty;
            var askSize = depth < book.Ask.Count ? $"{book.Ask[depth].Size:N0}" : string.Empty;
            var f = $"{bidSize, 10} {bidPrice, 10} {askPrice, 10} {askSize, 10}";
            t.WriteLine(f);
        }

        continue;
    }

    if (item.Data.Trade != null)
    {
        var trade = item.Data.Trade;
        var header = $"{ts:O} [{item.Data.Mapping}] [TRADE]";
        t.WriteLine();
        t.WriteLine(header);
        t.WriteLine($"{trade.Size}@{trade.Price} Id={trade.Id} Condition={trade.Condition}");
        continue;
    }
}